In [ ]:
#######################################
Function:
Make folders to direct specific outputs to.
    
Output:
1. Several folders for specific data storage
#######################################

In [26]:
# make log/bamfile/index/genome maps
mkdir log
mkdir bamfiles
mkdir indexes
mkdir genome
mkdir zfq
mkdir depth_out

mkdir: cannot create directory ‘log’: File exists
mkdir: cannot create directory ‘bamfiles’: File exists
mkdir: cannot create directory ‘indexes’: File exists
mkdir: cannot create directory ‘genome’: File exists


In [ ]:
#######################################
Input:
1. *fastq.gz (Untrimmed read files in FASTQ.gz format) 

Function:
Create a loop.lst file with input data for trimming.
    
Output:
1. loop.lst (TXT file with input data like directionaries and filenames)
#######################################

In [16]:
#prepare data loop.lst to loop through with tools
fqdir=/input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX/
ls ${fqdir} | egrep "*gz">untrimmed.lst
cat untrimmed.lst | awk '{print $0, "/powerplant/workspace/hramzr/RNA_seq/trimmed",  
"/input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX",substr($1,1,3)}'>loop.lst
rm untrimmed.lst
cat loop.lst | awk '{print $0, "/workspace/hramzr/RNA_seq/zfq/"substr($0,1,34)}'>loop2.lst
cat loop2.lst

TB1_CB0LJANXX_CAGCGT_L002_R1.fastq.gz /powerplant/workspace/hramzr/RNA_seq/trimmed /input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX TB1 /workspace/hramzr/RNA_seq/zfq/TB1_CB0LJANXX_CAGCGT_L002_R1.fastq
TB2_CB0LJANXX_TCAGGA_L002_R1.fastq.gz /powerplant/workspace/hramzr/RNA_seq/trimmed /input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX TB2 /workspace/hramzr/RNA_seq/zfq/TB2_CB0LJANXX_TCAGGA_L002_R1.fastq
TB3_CB0LJANXX_TCTTAA_L002_R1.fastq.gz /powerplant/workspace/hramzr/RNA_seq/trimmed /input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX TB3 /workspace/hramzr/RNA_seq/zfq/TB3_CB0LJANXX_TCTTAA_L002_R1.fastq
TB4_CB0LJANXX_TTTATG_L002_R1.fastq.gz /powerplant/workspace/hramzr/RNA_seq/trimmed /input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX TB4 /workspace/hramzr/RNA_seq/zfq/TB4_CB0LJANXX_TTTATG_L002_R1.fastq
TB5_CB0LJANXX_AGATAG_L002_R1.fastq.gz /powerplant/worksp

In [ ]:
#######################################
Input:
1. loop.lst (TXT file with input data like directionaries and filenames)

Function:
Run Trimmomatic on the FASTQ.GZ files from loop.lst and filter them on quality.
    
Output:
1. trimmed files(Trimmed FASTQ files)
#######################################

In [131]:
#trim reads on quality and minlen for downstream analysis
TRIMMOPATH=/software/bioinformatics/Trimmomatic-0.36
while read -r FN TRPTH PTH TN
do  
    bsub -o log/trim.out -e log/trim.err \
    "java -jar ${TRIMMOPATH}/trimmomatic-0.36.jar \
    SE -phred33 \
    ${PTH}/${FN} \
    ${TRPTH}/${TN}.fq.gz \
    ILLUMINACLIP:${TRIMMOPATH}/adapters/TruSeq3-PE.fa:2:30:10 \
    SLIDINGWINDOW:4:15 \
    MINLEN:50"
done < loop.lst

Job <302428> is submitted to default queue <normal>.
Job <302429> is submitted to default queue <normal>.
Job <302430> is submitted to default queue <normal>.
Job <302431> is submitted to default queue <normal>.
Job <302432> is submitted to default queue <normal>.
Job <302433> is submitted to default queue <normal>.
Job <302434> is submitted to default queue <normal>.
Job <302435> is submitted to default queue <normal>.
Job <302436> is submitted to default queue <normal>.
Job <302437> is submitted to default queue <normal>.
Job <302438> is submitted to default queue <normal>.
Job <302439> is submitted to default queue <normal>.
Job <302440> is submitted to default queue <normal>.
Job <302441> is submitted to default queue <normal>.
Job <302442> is submitted to default queue <normal>.
Job <302443> is submitted to default queue <normal>.
Job <302444> is submitted to default queue <normal>.
Job <302445> is submitted to default queue <normal>.
Job <302446> is submitted to default queue <no

In [ ]:
#######################################
Input:
1. nz_trevally.fasta(reference FASTA file of trevally)

Function:
Index reference FASTA as prerequisite step for mapping.
    
Output:
1. nz_trevally.fasta.fai(indexed FASTA file)
#######################################

In [3]:
#index in STAR format pre-map step
module load STAR/2.6.1d
genomedir=/powerplant/workspace/hramzr/RNA_seq/genome
bsub -o log/index.out -e log/index.err \
"STAR --runThreadN 6 \
--runMode genomeGenerate \
--genomeDir ${genomedir} \
--genomeFastaFiles /powerplant/workspace/hramzr/RNA_seq/genome/nz_trevally.fasta"

Job <254019> is submitted to default queue <normal>.


In [ ]:
#######################################
Input:
1. loop2.lst(file with data pertaining to the trimmed FQ files, that will be used for mapping)
2. nz_trevally.fasta.fai(indexed FASTA file)

Function:
Map the trimmed FASTQ files against the trevally reference sequence and create tissue sample and individual specific BAM files.
    
Output:
1. [Tissue name][individual number]*.bam (bam files specific to trevally individual and tissue sample)
#######################################

In [136]:
#mapping

cp /input/genomic/fish/Pseudocaranx/georgianus/Transcriptome/AGRF_CAGRF15066_CB0LJANXX/*gz  /workspace/hramzr/RNA_seq/zfq
gunzip /workspace/hramzr/RNA_seq/zfq/*gz

module load STAR/2.6.1d
TRIMMOPATH=/software/bioinformatics/Trimmomatic-0.36
INDIR=/powerplant/workspace/hramzr/RNA_seq/genome/

while read -r FN TRPTH PTH TN FQ
do  
    bsub -o log/star.out -e log/star.err \
    "STAR --genomeDir ${INDIR} --readFilesIn ${FQ} --runThreadN 6 --outSAMtype BAM SortedByCoordinate --outFileNamePrefix bamfiles/${TN}_out"
done < loop2.lst


Job <303234> is submitted to default queue <normal>.
Job <303235> is submitted to default queue <normal>.
Job <303236> is submitted to default queue <normal>.
Job <303237> is submitted to default queue <normal>.
Job <303238> is submitted to default queue <normal>.
Job <303239> is submitted to default queue <normal>.
Job <303240> is submitted to default queue <normal>.
Job <303241> is submitted to default queue <normal>.
Job <303242> is submitted to default queue <normal>.
Job <303243> is submitted to default queue <normal>.
Job <303244> is submitted to default queue <normal>.
Job <303245> is submitted to default queue <normal>.
Job <303246> is submitted to default queue <normal>.
Job <303247> is submitted to default queue <normal>.
Job <303248> is submitted to default queue <normal>.
Job <303249> is submitted to default queue <normal>.
Job <303250> is submitted to default queue <normal>.
Job <303251> is submitted to default queue <normal>.
Job <303252> is submitted to default queue <no

In [ ]:
#######################################
Input:
1. bams.lst(list of all bamfiles)


Function:
Index all created bamfiles as a prerequisite step to depth calling.

Output:
1. *.bai (indexed bam files)
#######################################

In [137]:
#index bamfiles
module load samtools/1.7

bamdir=/workspace/hramzr/RNA_seq/bamfiles/
ls ${bamdir} | egrep "*.bam$">bams.lst
while read -r BAM
do
    bsub \
    -o log/index.out -e log/index.err \
    "samtools index ${bamdir}${BAM}"
done < bams.lst

Job <303725> is submitted to default queue <normal>.
Job <303726> is submitted to default queue <normal>.
Job <303727> is submitted to default queue <normal>.
Job <303728> is submitted to default queue <normal>.
Job <303729> is submitted to default queue <normal>.
Job <303730> is submitted to default queue <normal>.
Job <303731> is submitted to default queue <normal>.
Job <303732> is submitted to default queue <normal>.
Job <303733> is submitted to default queue <normal>.
Job <303734> is submitted to default queue <normal>.
Job <303735> is submitted to default queue <normal>.
Job <303736> is submitted to default queue <normal>.
Job <303737> is submitted to default queue <normal>.
Job <303738> is submitted to default queue <normal>.
Job <303739> is submitted to default queue <normal>.
Job <303740> is submitted to default queue <normal>.
Job <303741> is submitted to default queue <normal>.
Job <303742> is submitted to default queue <normal>.
Job <303743> is submitted to default queue <no

In [ ]:
#######################################
Input:
1. bams.lst(list of all bamfiles)


Function:
Call depth of all bamfiles within the sex gene region on scaffold 001800.
This is done to look at coverage metrics of the RNA transcripts in regards to the sex determination gene.

Output:
1. *_depth (files with depth per individual and tissue sample)
#######################################

In [138]:
module load samtools/1.7
bamdir=/workspace/hramzr/RNA_seq/bamfiles/

while read -r BAM
do
    bsub \
    -o log/dpth.out -e log/dpth.err \
    "samtools depth -a -r trevally001800:200-2400 ${bamdir}${BAM} > depth_out/${BAM}_depth" 
done < bams.lst

Job <303760> is submitted to default queue <normal>.
Job <303761> is submitted to default queue <normal>.
Job <303762> is submitted to default queue <normal>.
Job <303763> is submitted to default queue <normal>.
Job <303764> is submitted to default queue <normal>.
Job <303765> is submitted to default queue <normal>.
Job <303766> is submitted to default queue <normal>.
Job <303767> is submitted to default queue <normal>.
Job <303768> is submitted to default queue <normal>.
Job <303769> is submitted to default queue <normal>.
Job <303770> is submitted to default queue <normal>.
Job <303771> is submitted to default queue <normal>.
Job <303772> is submitted to default queue <normal>.
Job <303773> is submitted to default queue <normal>.
Job <303774> is submitted to default queue <normal>.
Job <303775> is submitted to default queue <normal>.
Job <303776> is submitted to default queue <normal>.
Job <303777> is submitted to default queue <normal>.
Job <303778> is submitted to default queue <no

In [ ]:
#######################################
Input:
1. bams2.lst(list of all bamfiles)

Function:
Print out an overview of transcripts that have a higher coverage than 0 per base on scaffold 001800 region 200-2400.

Output:
1. Bash output (overview of transcripts within sex determination gene region)

Analysis conclusion:
Coverage too slim to consider this as present, 2 coverage is slim, and over 19 bases on just one tissue sample and individual.
== Gene NOT present on input tissue samples
#######################################

In [141]:
cat bams.lst | awk '{print $1, substr($1,1,3)}'>bams2.lst
while read -r BAM ID
do
    cat depth_out/${BAM}_depth | awk '{if ($3>0) print $0, a}' a=${ID}
done < bams2.lst

trevally001800	255	2 TH1
trevally001800	256	2 TH1
trevally001800	257	2 TH1
trevally001800	258	2 TH1
trevally001800	259	2 TH1
trevally001800	260	2 TH1
trevally001800	261	2 TH1
trevally001800	262	2 TH1
trevally001800	1097	2 TH1
trevally001800	1098	2 TH1
trevally001800	1099	2 TH1
trevally001800	1100	2 TH1
trevally001800	1101	2 TH1
trevally001800	1102	2 TH1
trevally001800	1103	2 TH1
trevally001800	1104	2 TH1
trevally001800	1105	2 TH1
trevally001800	1106	2 TH1
trevally001800	1107	2 TH1
